# Đánh giá Mô hình Tiếng Việt có dấu

Notebook này thực hiện các bước để đánh giá mô hình ngôn ngữ Tiếng Việt đã được huấn luyện. Chúng ta sẽ:
1. Tải dữ liệu và chia thành tập huấn luyện và tập kiểm tra.
2. Tải mô hình đã huấn luyện.
3. Tính toán các độ đo đánh giá:
    - Perplexity.
    - Độ chính xác phục hồi dấu.
    - Độ chính xác Top-3 phục hồi dấu.
4. Trực quan hóa kết quả.

In [1]:
import nltk

# Download the standard 'punkt' resource (already in notebook)
nltk.download('punkt')

# Download the 'punkt_tab' resource which appears to be missing
nltk.download('punkt_tab')

# Import các thư viện cần thiết
import os
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Import các module từ project của bạn
# Đảm bảo rằng PYTHONPATH được thiết lập đúng hoặc notebook này nằm ở thư mục gốc của project
import utils.data_loader as data_loader
import utils.model_trainer as model_trainer
from utils.utils import remove_vn_accent # Cần cho việc tạo từ không dấu để kiểm tra
# from sklearn.model_selection import train_test_split # Already imported above

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tontide1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tontide1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tontide1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Tải và Chuẩn bị Dữ liệu

Tải corpus và chia thành 80% cho tập huấn luyện và 20% cho tập kiểm tra.
Lưu ý: Mô hình NLTK thường được huấn luyện trên một danh sách các câu, mỗi câu là một danh sách các token.
Việc chia dữ liệu ở đây chủ yếu là để dành một phần dữ liệu gốc cho việc đánh giá các tác vụ cụ thể như phục hồi dấu, không nhất thiết dùng để huấn luyện lại mô hình trong notebook này.

In [3]:
# Tải toàn bộ corpus (danh sách các câu, mỗi câu là danh sách các từ)
print("Đang tải corpus...")
# Giả sử load_corpus trả về một danh sách các câu (list of lists of strings)
# Ví dụ: [['tôi', 'đi', 'học'], ['trời', 'đẹp']]
full_corpus_sentences = data_loader.load_corpus()

print(f"Đã tải {len(full_corpus_sentences)} câu từ corpus.")

if not full_corpus_sentences:
    print("Không tải được corpus. Vui lòng kiểm tra lại data_loader.py và đường dẫn dữ liệu.")
else:
    print(f"Tải thành công {len(full_corpus_sentences)} câu.")

    # Chia dữ liệu: 80% train, 20% test
    # Chúng ta sẽ chia danh sách các câu
    train_sentences, test_sentences = train_test_split(full_corpus_sentences, test_size=0.2, random_state=42)

    print(f"Số câu trong tập huấn luyện: {len(train_sentences)}")
    print(f"Số câu trong tập kiểm tra: {len(test_sentences)}")

    # Để tính perplexity, mô hình NLTK cần một danh sách các từ (flattened list) hoặc một iterable của các n-grams từ test_sentences
    # Tạo test_data cho perplexity: một list các token từ test_sentences
    test_data_tokens = [token for sentence in test_sentences for token in sentence]
    print(f"Tổng số token trong tập kiểm tra (dùng cho perplexity): {len(test_data_tokens)}")

Đang tải corpus...
Loading corpus from: c:\Users\tontide1\Desktop\ML_ML\data\Train_Full


Reading files in Train_Full: 0it [00:00, ?it/s]
Reading files in Van hoa: 100%|██████████| 5250/5250 [00:00<00:00, 14441.77it/s]


Loaded 42744 documents.
Processing 1208363 raw sentences...


Tokenizing sentences:  26%|██▌       | 312893/1208363 [00:28<01:22, 10862.97it/s]


KeyboardInterrupt: 

## 2. Tải Mô hình đã Huấn luyện

Tải mô hình đã được huấn luyện từ tệp `.pkl`.

In [ ]:
MODEL_FILENAME = "models/kneserney_trigram_model.pkl" # Hoặc tên tệp mô hình của bạn

if os.path.exists(MODEL_FILENAME):
    print(f"Đang tải mô hình từ {MODEL_FILENAME}...")
    try:
        with open(MODEL_FILENAME, 'rb') as f:
            model = pickle.load(f)
        print("Tải mô hình thành công.")
    except Exception as e:
        print(f"Lỗi khi tải mô hình: {e}")
        model = None
else:
    print(f"Không tìm thấy tệp mô hình tại {MODEL_FILENAME}. Vui lòng huấn luyện mô hình trước.")
    model = None

# In thông tin cơ bản về mô hình nếu tải thành công
if model:
    print(f"Model type: {type(model)}")
    if hasattr(model, 'order'):
        print(f"Model order (N-gram): {model.order}")

Đang tải mô hình từ models/kneserney_trigram_model.pkl...
Tải mô hình thành công.
Model type: <class 'nltk.lm.models.KneserNeyInterpolated'>
Model order (N-gram): 3


## 3. Định nghĩa và Tính toán các Độ đo Đánh giá

Chúng ta sẽ sử dụng các độ đo sau:
1.  **Perplexity**: Đo lường mức độ "ngạc nhiên" của mô hình khi gặp một chuỗi từ mới. Giá trị càng thấp càng tốt.
2.  **Độ chính xác Phục hồi Dấu (Accent Restoration Accuracy)**: Tỷ lệ các từ được phục hồi dấu chính xác.
3.  **Độ chính xác Top-3 Phục hồi Dấu (Top-3 Accent Restoration Accuracy)**: Tỷ lệ các từ mà dạng đúng có dấu nằm trong 3 dự đoán hàng đầu của mô hình.

**LƯU Ý QUAN TRỌNG VỀ DỰ ĐOÁN PHỤC HỒI DẤU:**
Để tính toán độ chính xác phục hồi dấu, chúng ta cần một hàm có khả năng:
- Nhận một từ không dấu (và có thể là ngữ cảnh xung quanh).
- Trả về từ có dấu được dự đoán bởi mô hình (hoặc danh sách N dự đoán tốt nhất).

Hàm này có thể cần được xây dựng trong `utils/predictor.py`. Ví dụ, nó có thể sử dụng mô hình ngôn ngữ để tính xác suất của các biến thể có dấu khác nhau của một từ không dấu và chọn biến thể có xác suất cao nhất.

**Giả định:**
Hiện tại, chúng ta sẽ giả định có một hàm `predict_top_n_accented(unaccented_word, context, model, n=1)` trong `utils.predictor` hoặc sẽ mô phỏng một cách đơn giản nếu chưa có.
Đối với mô hình N-gram của NLTK, việc trực tiếp lấy "dự đoán từ có dấu" từ một từ không dấu không phải là một tính năng có sẵn. Mô hình N-gram tính xác suất của một chuỗi từ.
Bạn có thể cần kết hợp `remove_vn_accent` và `gen_accents_word` từ `utils.utils` cùng với khả năng tính score của mô hình NLTK để xây dựng hàm predictor này.

Ví dụ về cách `predictor` có thể hoạt động:
1. Cho từ không dấu `w_no_accent`.
2. Sử dụng `gen_accents_word(w_no_accent)` để lấy tất cả các biến thể có dấu `w_accented_variants`.
3. Đối với mỗi `variant` trong `w_accented_variants`, tính điểm/xác suất của nó trong ngữ cảnh hiện tại bằng cách sử dụng `model.score(variant, context)`.
4. Chọn variant có điểm cao nhất.

Do sự phức tạp này, phần code dưới đây cho phục hồi dấu sẽ mang tính giả định cao về sự tồn tại của hàm `predict_top_n_accented_words`.

In [ ]:
# --- 3.1. Perplexity ---
perplexity_score = -1
if model and test_data_tokens:
    try:
        # NLTK model.perplexity() mong muốn một iterable của các token hoặc n-grams
        # Đảm bảo test_data_tokens được padding đúng cách nếu cần cho model.vocab
        # Hoặc sử dụng một cách tiếp cận phù hợp với cách model của bạn tính perplexity
        # Một số mô hình NLTK có thể cần test_data được xử lý thành n-grams trước
        # Ví dụ: list(nltk.ngrams(test_data_tokens, model.order))
        # Tuy nhiên, nhiều mô hình chấp nhận danh sách token phẳng và tự xử lý.

        # Lọc các token không có trong vocab của model để tránh lỗi (OOV - Out Of Vocabulary)
        # Hoặc đảm bảo model của bạn xử lý OOV (ví dụ, bằng smoothing)
        # test_data_for_perplexity = [token for token in test_data_tokens if token in model.vocab]
        # if not test_data_for_perplexity:
        #    print("Cảnh báo: Không có token nào trong test_data_tokens thuộc vocab của model. Không thể tính perplexity.")
        # else:
        #    perplexity_score = model.perplexity(test_data_for_perplexity)

        # Cách tiếp cận đơn giản hơn, giả sử model.perplexity xử lý OOV tokens
        # hoặc chúng ta chấp nhận kết quả có thể bị ảnh hưởng bởi OOV
        print(f"Đang tính Perplexity trên {len(test_data_tokens)} token...")
        # Để tính perplexity chính xác, test_data_tokens nên được chuẩn bị theo cách model đã được huấn luyện
        # (ví dụ, có thể cần thêm <s>, </s> markers nếu model được huấn luyện với chúng)
        # Vì load_corpus không thêm các markers này, chúng ta sẽ tính trên raw tokens.
        # Điều này có thể không hoàn toàn chính xác nếu model mong đợi padding.

        # Chuyển đổi test_sentences thành dạng mà perplexity có thể tính toán
        # Thường là một generator của các n-gram hoặc một list các token
        # Ví dụ, nếu model là trigram:
        # test_ngrams = [nltk.trigrams(sent) for sent in test_sentences]
        # test_sequences = (ngram for sent_ngrams in test_ngrams for ngram in sent_ngrams)
        # perplexity_score = model.perplexity(test_sequences)

        # Cách đơn giản nhất là truyền list of tokens, nhiều model NLTK hỗ trợ điều này.
        perplexity_score = model.perplexity(test_data_tokens)
        print(f"Perplexity trên tập kiểm tra: {perplexity_score:.4f}")

    except Exception as e:
        print(f"Lỗi khi tính perplexity: {e}")
        print("Lưu ý: Tính perplexity có thể phức tạp tùy thuộc vào cách model được huấn luyện và xử lý OOV.")
        perplexity_score = float('inf') # Hoặc giá trị lỗi khác
else:
    print("Mô hình chưa được tải hoặc không có dữ liệu kiểm tra để tính perplexity.")
    perplexity_score = float('inf')

Đang tính Perplexity trên 4313232 token...
Lỗi khi tính perplexity: 'KneserNey' object has no attribute '_order'
Lưu ý: Tính perplexity có thể phức tạp tùy thuộc vào cách model được huấn luyện và xử lý OOV.


In [ ]:
perplexity_score = -1
if model and test_sentences: # Sử dụng test_sentences để chuẩn bị dữ liệu có padding
    try:
        if not hasattr(model, 'order'):
            raise AttributeError("Model không có thuộc tính 'order', không thể xác định padding cho perplexity.")

        print(f"Model order (N-gram): {model.order}")
        print("Đang chuẩn bị dữ liệu kiểm tra cho perplexity với padding...")

        padded_test_corpus_tokens = []
        for sent in test_sentences:
            # Áp dụng padding giống như khi huấn luyện
            # Tham số 'n' cho pad_both_ends là bậc của N-gram (model.order)
            # Mặc định left_pad_symbol="<s>", right_pad_symbol="</s>"
            padded_sent = list(nltk.lm.preprocessing.pad_both_ends(sent, n=model.order))
            # print(padded_sent)
            padded_test_corpus_tokens.extend(padded_sent)

        if not padded_test_corpus_tokens:
            print("Cảnh báo: Corpus kiểm tra sau khi padding bị rỗng.")
            perplexity_score = float('inf')
        else:
            print(f"Đang tính Perplexity trên {len(padded_test_corpus_tokens)} tokens đã được padding...")
            perplexity_score = model.perplexity(padded_test_corpus_tokens)
            print(f"Perplexity trên tập kiểm tra: {perplexity_score:.4f}")

    except AttributeError as ae:
        print(f"Lỗi thuộc tính khi chuẩn bị/tính perplexity: {ae}")
        perplexity_score = float('inf')
    except Exception as e:
        print(f"Lỗi khi tính perplexity: {e}")
        print("Lưu ý: Tính perplexity có thể phức tạp tùy thuộc vào cách model được huấn luyện và xử lý OOV.")
        perplexity_score = float('inf')
else:
    print("Mô hình chưa được tải hoặc không có dữ liệu kiểm tra (test_sentences) để tính perplexity.")
    perplexity_score = float('inf')

Model order (N-gram): 3
Đang chuẩn bị dữ liệu kiểm tra cho perplexity với padding...
Đang tính Perplexity trên 5114532 tokens đã được padding...
Lỗi thuộc tính khi chuẩn bị/tính perplexity: 'KneserNey' object has no attribute '_order'


### Giả định hàm `predict_top_n_accented_words`
Do chưa có hàm `predict_top_n_accented_words` trong `utils.predictor`, chúng ta sẽ tạo một hàm giả định ở đây để minh họa.
Trong thực tế, bạn cần xây dựng hàm này một cách cẩn thận dựa trên mô hình ngôn ngữ của mình.

In [ ]:
# --- Giả định hàm predictor ---
# Đây là một hàm GIẢ ĐỊNH. Bạn cần thay thế bằng logic thực tế của mình.
# Hàm này nên nằm trong utils/predictor.py
def predict_top_n_accented_words_mock(unaccented_word: str, context: list[str], model_lm, n: int = 3) -> list[str]:
    """
    Hàm MOCK để dự đoán N từ có dấu khả năng nhất cho một từ không dấu.
    Trong thực tế, hàm này sẽ sử dụng model_lm để tính điểm các biến thể có dấu.
    """
    # Bước 1: Tạo các biến thể có dấu (sử dụng utils.utils.gen_accents_word nếu có)
    # Giả sử gen_accents_word tồn tại và hoạt động tốt
    try:
        from utils.utils import gen_accents_word
        possible_accented_words = list(gen_accents_word(unaccented_word))
    except ImportError: # Fallback nếu không import được
        # Tạo một vài biến thể giả định đơn giản
        if unaccented_word == "hoc":
            possible_accented_words = ["học", "hóc", "hộc"]
        elif unaccented_word == "troi":
            possible_accented_words = ["trời", "trôi", "trối"]
        elif unaccented_word == "dep":
            possible_accented_words = ["đẹp", "dẹp"]
        else: # Trả về chính từ không dấu nếu không có quy tắc
            possible_accented_words = [unaccented_word]

    if not possible_accented_words:
        return [unaccented_word] # Trả về từ gốc nếu không có biến thể

    # Bước 2: Tính điểm cho mỗi biến thể (đây là phần phức tạp cần model_lm)
    # Vì đây là mock, chúng ta sẽ trả về ngẫu nhiên hoặc dựa trên thứ tự đơn giản
    # Trong thực tế: scores = [model_lm.score(variant, context) for variant in possible_accented_words]
    # sorted_variants = [v for _, v in sorted(zip(scores, possible_accented_words), reverse=True)]

    # Mock logic: chỉ trả về tối đa N phần tử đầu tiên từ list (hoặc ngẫu nhiên)
    random.shuffle(possible_accented_words) # Giả vờ là đã sắp xếp theo score
    return possible_accented_words[:n]

# --- 3.2. & 3.3. Độ chính xác Phục hồi Dấu ---
correct_predictions = 0
top_n_correct_predictions = 0
total_words_evaluated = 0
N_TOP = 3 # Cho Top-N accuracy

accent_restoration_results = [] # Lưu trữ (original, unaccented, predicted, is_correct, is_in_top_n)

if model and test_sentences:
    print(f"Đang đánh giá độ chính xác phục hồi dấu trên {len(test_sentences)} câu kiểm tra...")
    for i, sentence in enumerate(test_sentences):
        if i % 100 == 0 and i > 0: # In tiến trình
            print(f"Đã xử lý {i}/{len(test_sentences)} câu...")

        context = [] # Ngữ cảnh cho mô hình N-gram (ví dụ: N-1 từ trước đó)
        for original_word in sentence:
            if not original_word.strip() or not any(c.isalpha() for c in original_word): # Bỏ qua từ rỗng hoặc không có chữ cái
                context.append(original_word) # Vẫn thêm vào context
                continue

            unaccented_word = remove_vn_accent(original_word)

            # Lấy dự đoán từ mô hình (sử dụng hàm mock)
            # Trong thực tế, context sẽ là N-1 từ đứng trước `original_word`
            # Ví dụ, nếu model là trigram (order=3), context là 2 từ trước đó.
            # current_context = context[-(model.order - 1):] if hasattr(model, 'order') else []
            predicted_top_n = predict_top_n_accented_words_mock(unaccented_word, context, model, n=N_TOP)

            predicted_word = ""
            is_correct = False
            is_in_top_n = False

            if predicted_top_n:
                predicted_word = predicted_top_n[0] # Dự đoán tốt nhất
                if predicted_word == original_word:
                    correct_predictions += 1
                    is_correct = True

                if original_word in predicted_top_n:
                    top_n_correct_predictions += 1
                    is_in_top_n = True

            accent_restoration_results.append({
                "original": original_word,
                "unaccented": unaccented_word,
                "predicted_top1": predicted_word,
                "predicted_topN": predicted_top_n,
                "is_correct_top1": is_correct,
                "is_in_topN": is_in_top_n
            })
            total_words_evaluated += 1
            context.append(original_word) # Cập nhật context cho từ tiếp theo

    if total_words_evaluated > 0:
        accuracy = (correct_predictions / total_words_evaluated) * 100
        top_n_accuracy = (top_n_correct_predictions / total_words_evaluated) * 100
        print(f"Tổng số từ đã đánh giá (phục hồi dấu): {total_words_evaluated}")
        print(f"Độ chính xác phục hồi dấu (Accuracy): {accuracy:.2f}%")
        print(f"Độ chính xác Top-{N_TOP} phục hồi dấu: {top_n_accuracy:.2f}%")
    else:
        print("Không có từ nào được đánh giá cho việc phục hồi dấu.")
        accuracy = 0
        top_n_accuracy = 0
else:
    print("Mô hình chưa được tải hoặc không có dữ liệu kiểm tra để đánh giá phục hồi dấu.")
    accuracy = 0
    top_n_accuracy = 0

# Chuyển kết quả chi tiết thành DataFrame để dễ xem
df_results = pd.DataFrame(accent_restoration_results)

Đang đánh giá độ chính xác phục hồi dấu trên 200325 câu kiểm tra...
Đã xử lý 100/200325 câu...
Đã xử lý 200/200325 câu...
Đã xử lý 300/200325 câu...
Đã xử lý 400/200325 câu...
Đã xử lý 500/200325 câu...
Đã xử lý 600/200325 câu...


### Xem một vài ví dụ về kết quả phục hồi dấu

In [ ]:
if not df_results.empty:
    print("Một vài ví dụ về kết quả phục hồi dấu (Top-1):")
    print(df_results[df_results['original'] != df_results['predicted_top1']].sample(min(10, len(df_results[df_results['original'] != df_results['predicted_top1']]))))

    print(f"Một vài ví dụ dự đoán Top-1 đúng:")
    print(df_results[df_results['is_correct_top1'] == True].sample(min(5, len(df_results[df_results['is_correct_top1'] == True]))))

    print(f"Một vài ví dụ dự đoán Top-1 sai nhưng đúng trong Top-{N_TOP}:")
    print(df_results[(df_results['is_correct_top1'] == False) & (df_results['is_in_topN'] == True)].sample(min(5, len(df_results[(df_results['is_correct_top1'] == False) & (df_results['is_in_topN'] == True)]))))
else:
    print("Không có kết quả chi tiết để hiển thị.")

## 4. Trực quan hóa Kết quả

Vẽ biểu đồ thể hiện các độ đo đã tính toán.

In [ ]:
metrics_names = ['Perplexity', f'Accuracy (Top-1)', f'Accuracy (Top-{N_TOP})']
# Perplexity có thang đo khác, nên có thể không vẽ chung hoặc cần chuẩn hóa
# Ở đây chúng ta sẽ vẽ Accuracy riêng
accuracy_scores = [accuracy, top_n_accuracy]
accuracy_labels = [f'Accuracy (Top-1)', f'Accuracy (Top-{N_TOP})']

plt.figure(figsize=(12, 6))

# Biểu đồ cho Perplexity (nếu có giá trị hợp lệ)
if perplexity_score != float('inf') and perplexity_score != -1:
    plt.subplot(1, 2, 1)
    sns.barplot(x=["Perplexity"], y=[perplexity_score])
    plt.title(f'Perplexity Score: {perplexity_score:.2f}')
    plt.ylabel("Giá trị Perplexity (càng thấp càng tốt)")
else:
    print("Không có giá trị Perplexity hợp lệ để vẽ.")
    # Tạo một plot trống nếu không có perplexity
    plt.subplot(1, 2, 1)
    plt.text(0.5, 0.5, 'Perplexity N/A', ha='center', va='center', fontsize=12)
    plt.title('Perplexity Score')
    plt.xticks([])
    plt.yticks([])


# Biểu đồ cho Độ chính xác Phục hồi Dấu
plt.subplot(1, 2, 2)
if total_words_evaluated > 0:
    bars = sns.barplot(x=accuracy_labels, y=accuracy_scores, palette="viridis")
    plt.title('Độ chính xác Phục hồi Dấu')
    plt.ylabel('Tỷ lệ (%)')
    plt.ylim(0, 100) # Giới hạn trục y từ 0 đến 100%
    for bar in bars.patches:
        plt.text(bar.get_x() + bar.get_width() / 2,
                 bar.get_height() + 0.5,
                 f'{bar.get_height():.2f}%',
                 ha='center', va='bottom')
else:
    plt.text(0.5, 0.5, 'Accuracy N/A', ha='center', va='center', fontsize=12)
    plt.title('Độ chính xác Phục hồi Dấu')
    plt.xticks([])
    plt.yticks([])


plt.tight_layout()
plt.show()

## 5. Phân tích Lỗi (Error Analysis) - Tùy chọn

Xem xét các trường hợp mô hình dự đoán sai để hiểu rõ hơn về điểm yếu của nó.

In [ ]:
if not df_results.empty:
    # Các trường hợp Top-1 dự đoán sai
    errors_df = df_results[df_results['is_correct_top1'] == False]
    print(f"Số lượng dự đoán sai (Top-1): {len(errors_df)} / {total_words_evaluated}")

    if not errors_df.empty:
        print("10 ví dụ ngẫu nhiên về các lỗi dự đoán (Top-1):")
        # Hiển thị 'original', 'unaccented', 'predicted_top1'
        print(errors_df[['original', 'unaccented', 'predicted_top1']].sample(min(10, len(errors_df))))

        # Thống kê các lỗi phổ biến (ví dụ: những từ nào hay bị sai nhất)
        # (Phần này có thể phức tạp hơn và tùy thuộc vào nhu cầu)
        common_errors = errors_df.groupby(['original', 'predicted_top1']).size().sort_values(ascending=False)
        print("Các cặp (từ gốc, từ dự đoán sai) phổ biến nhất:")
        print(common_errors.head(10))
    else:
        print("Không có lỗi nào trong dự đoán Top-1 (hoặc không có từ nào được đánh giá).")
else:
    print("Không có kết quả chi tiết để phân tích lỗi.")

## 6. Kết luận và Hướng Phát triển Tiếp theo

Dựa trên kết quả đánh giá:
- Mô hình hoạt động tốt như thế nào?
- Các điểm mạnh và điểm yếu là gì?
- Có thể cải thiện mô hình bằng cách nào (ví dụ: thêm dữ liệu, thay đổi kiến trúc mô hình, tinh chỉnh tham số, cải thiện hàm predictor)?

(Điền nhận xét của bạn vào đây)